In [1]:
import pandas as pd

In [3]:
# import the CSV files
tabelog_data = pd.read_csv('tabelog_data_geocoded.csv')
tripadvisor_data = pd.read_csv('trip_advisor_data_geocoded.csv')

In [6]:
# inspect the data
tabelog_data.head()

,Ranking,Name,URL,Rating,Price Range,Review Count,Nearest Station,Categories,Address,Reservation Only,Latitude,Longitude
0,1,Shimbashi Hoshino,https://tabelog.com/en/tokyo/A1314/A131401/131...,4.65,50000-59999,275,Onarimon Sta.,Japanese Cuisine,東京都港区新橋5-31-3,True,35.662458,139.755308
1,2,Nihonbashi Kakigaracho Sugita,https://tabelog.com/en/tokyo/A1302/A130204/130...,4.64,40000-49999,897,Suitengumae Sta.,Sushi,東京都中央区日本橋蛎殻町1-33-6 ビューハイツ日本橋 B1F,True,35.681852,139.784998
2,3,aca,https://tabelog.com/en/tokyo/A1302/A130202/132...,4.63,60000-79999,523,Mitsukoshimae Sta.,Spain,東京都中央区日本橋室町2-1-1 三井2号館,True,35.686864,139.772481
3,4,Higashiazabu Amamoto,https://tabelog.com/en/tokyo/A1314/A131401/131...,4.62,60000-79999,572,Akabanebashi Sta.,Sushi,東京都港区東麻布1-7-9 ザ・ソノビル 102,True,35.657349,139.743468
4,5,Matsukawa,https://tabelog.com/en/tokyo/A1307/A130701/131...,4.61,80000-99999,509,Roppongi Itchome Sta.,Japanese Cuisine,東京都港区赤坂1-11-6 赤坂テラスハウス １階,True,35.667462,139.742390


In [7]:
tripadvisor_data.head()

,Ranking,Name,URL,Rating,Price Range,Review Count,Categories,Address,Latitude,Longitude
0,1,Gyopao Gyoza Roppongi,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,Chinese,"4-9-8, Roppongi, Minato 106-0032 Tokyo Prefecture",35.663587,139.732158
1,2,Sushi Koshikawa,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,Japanese,"5-4-14 Trade Akasaka Bldg. 2F, Akasaka, Minato...",35.673409,139.731514
2,3,Teppanyaki Sumiyaki Saito,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,Japanese,"6 Chome−1−6 Zakusen Building 6F, Roppongi, Min...",35.662490,139.732504
3,4,Rokkasen Otakibashiidori,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,Seafood,"7 Chome−2−6 Nishishinjuku K-1 Bldg. B1F, Nishi...",35.695454,139.698683
4,5,Premium Sake Pub Gashue,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,Brew Pub,"2-13-5 Higashiueno, Taito 110-0015 Tokyo Prefe...",35.709440,139.777973


In [8]:
# group the data by cuisine and count the number of restaurants
tabelog_grouped = tabelog_data.groupby('Categories').size().reset_index(name='Tabelog_Count')
tripadvisor_grouped = tripadvisor_data.groupby('Categories').size().reset_index(name='Tripadvisor_Count')

In [14]:
# combine the grouped data into a single dataframe
# perform an outer join to ensure all categories from both datasets are included

combined_data = pd.merge(
    tabelog_grouped,
    tripadvisor_grouped,
    on='Categories',
    how='outer'
)

combined_data.fillna(0, inplace=True)

combined_data.head()

,Categories,Tabelog_Count,Tripadvisor_Count
0,$$$$,0.0,1.0
1,American,0.0,9.0
2,Bar,1.0,15.0
3,Barbecue,0.0,3.0
4,Beef dishes,2.0,0.0


In [15]:
# convert counts to integers (after filling NaNs with 0s)
combined_data['Tabelog_Count'] = combined_data['Tabelog_Count'].astype(int)
combined_data['Tripadvisor_Count'] = combined_data['Tripadvisor_Count'].astype(int)

combined_data.head()

,Categories,Tabelog_Count,Tripadvisor_Count
0,$$$$,0,1
1,American,0,9
2,Bar,1,15
3,Barbecue,0,3
4,Beef dishes,2,0


In [16]:
# adding a total count column in case I want to use it 
combined_data['Total_Count'] = combined_data['Tabelog_Count'] + combined_data['Tripadvisor_Count']
combined_data = combined_data.sort_values(by='Total_Count', ascending=False)

combined_data.head()

,Categories,Tabelog_Count,Tripadvisor_Count,Total_Count
27,Japanese,0,119,119
39,Sushi,43,4,47
28,Japanese Cuisine,41,0,41
16,French,27,2,29
2,Bar,1,15,16


In [22]:
# export the cleaned data to a new CSV file
output_csv = 'cleaned_cuisines_by_platform.csv'
combined_data.to_csv(output_csv, index=False)

In [23]:
combined_data

,Categories,Tabelog_Count,Tripadvisor_Count,Total_Count
27,Japanese,0,119,119
39,Sushi,43,4,47
28,Japanese Cuisine,41,0,41
16,French,27,2,29
2,Bar,1,15,16
11,Chinese,10,4,14
26,Italian,7,7,14
44,Yakiniku (BBQ Beef),13,0,13
23,Innovative,10,0,10
45,Yakitori (Grilled chicken skewers),10,0,10


In [31]:
# doesn't look right. lets clean the data a bit.
import re

csv_file = 'tabelog_data_geocoded.csv'
df = pd.read_csv(csv_file)

# clean the 'Categories' column
# a) Remove all text within parentheses () and strip whitespaces
df['Categories'] = df['Categories'].apply(lambda x: re.sub(r'\s*\([^)]*\)', '', str(x)).strip())

# relabel 'Innovative' as 'Fusion'
df['Categories'] = df['Categories'].replace('Innovation', 'Fusion')

# relabel 'Japanese Cuisine' as 'Kaiseki'
df['Categories'] = df['Categories'].replace('Japanese Cuisine', 'Kaiseki')

# save the cleaned data to a new CSV file
output_csv = 'tabelog_data_cleaned.csv'
df.to_csv(output_csv, index=False)

print(f"Data cleaned and saved to {output_csv}")

Data cleaned and saved to tabelog_data_cleaned.csv


In [33]:
# import the CSV files
tabelog_data = pd.read_csv('tabelog_data_cleaned.csv')
tripadvisor_data = pd.read_csv('trip_advisor_data_geocoded.csv')

tabelog_grouped = tabelog_data.groupby('Categories').size().reset_index(name='Tabelog_Count')
tripadvisor_grouped = tripadvisor_data.groupby('Categories').size().reset_index(name='Tripadvisor_Count')

combined_cleaned = pd.merge(
    tabelog_grouped,
    tripadvisor_grouped,
    on='Categories',
    how='outer'
)

combined_cleaned.fillna(0, inplace=True)

combined_cleaned['Total_Count'] = combined_cleaned['Tabelog_Count'] + combined_cleaned['Tripadvisor_Count']
combined_cleaned = combined_cleaned.sort_values(by='Total_Count', ascending=False)

combined_cleaned

,Categories,Tabelog_Count,Tripadvisor_Count,Total_Count
27,Japanese,0.0,119.0,119.0
39,Sushi,43.0,4.0,47.0
28,Kaiseki,41.0,0.0,41.0
16,French,27.0,2.0,29.0
2,Bar,1.0,15.0,16.0
11,Chinese,10.0,4.0,14.0
26,Italian,7.0,7.0,14.0
44,Yakiniku,13.0,0.0,13.0
23,Innovative,10.0,0.0,10.0
45,Yakitori,10.0,0.0,10.0


In [38]:
csv_file = 'trip_advisor_data_geocoded.csv'
df = pd.read_csv(csv_file)

# Step 2: Count rows labeled as 'Japanese' before the edit
pre_edit_japanese_count = df[df['Categories'] == 'Japanese'].shape[0]

# Step 3: Define the mappings for category updates
name_to_category_mapping = {
    'sumo': 'Tourist',
    'ninja': 'Tourist',
    'sushi': 'Sushi',
    'teppanyaki': 'Teppanyaki',
    'teppan': 'Teppanyaki',
    'mo-mo-paradise': 'Sukiyaki',
    'sukiyaki': 'Sukiyaki',
    'ramen': 'Ramen',
    'tantan': 'Ramen',
    'ichiran': 'Ramen',
    'monja': 'Monjayaki',
    'shabu': 'Shabu-Shabu',
    'nabezo': 'Shabu-Shabu',
    'yakiniku': 'Yakiniku',
    'gyukatsu': 'Gyukatsu',
    'kobe': 'Wagyu',
    'tendon': 'Tempura',
    'yakitori': 'Yakitori',
    'tonkatsu': 'Tonkatsu',
    'kushiyaki': 'Kushiyaki',
    'nobu': 'Sushi'
}

# Step 4: Update the 'Categories' column based on the 'Name' column
def update_categories(row):
    name = str(row['Name']).lower()  # Convert to lowercase for case-insensitive matching
    category = row['Categories']
    
    # Process rows labeled 'Japanese' or 'Japanese Cuisine'
    if category in ['Japanese', 'Japanese Cuisine']:
        for keyword, mapped_category in name_to_category_mapping.items():
            if keyword in name:
                return mapped_category
    
    # If the name contains 'sumo' or 'ninja', override the category regardless of current value
    if 'sumo' in name or 'ninja' in name:
        return 'Tourist'
    
    # Return the original category if no changes are needed
    return category

# Apply the function to update the 'Categories' column
df['Categories'] = df.apply(update_categories, axis=1)

# Step 5: Count rows still labeled as 'Japanese' after the edit
post_edit_japanese_count = df[df['Categories'] == 'Japanese'].shape[0]

# Step 6: Save the cleaned data to a new CSV file
output_csv = 'trip_advisor_data_cleaned.csv'
df.to_csv(output_csv, index=False)

# Print the counts
print(f"Rows labeled as 'Japanese' before edits: {pre_edit_japanese_count}")
print(f"Rows still labeled as 'Japanese' after edits: {post_edit_japanese_count}")
print(f"Cleaned data saved to {output_csv}.")


Rows labeled as 'Japanese' before edits: 119
Rows still labeled as 'Japanese' after edits: 56
Cleaned data saved to trip_advisor_data_cleaned.csv.


In [40]:
csv_file = 'trip_advisor_data_cleaned.csv'  # Replace with your cleaned file name
df = pd.read_csv(csv_file)

# Step 2: Filter rows still labeled as 'Japanese'
remaining_japanese_rows = df[df['Categories'] == 'Japanese']
remaining_japanese_count = remaining_japanese_rows.shape[0]

# Step 3: Print the count and details of remaining rows
print(f"Rows still labeled as 'Japanese': {remaining_japanese_count}")
if remaining_japanese_count > 0:
    print("\nDetails of rows still labeled as 'Japanese':")
    print(remaining_japanese_rows.to_string(index=False))  # Print rows without the index

# Step 4: Save the remaining 'Japanese' rows to a separate file (optional)
output_file = 'remaining_japanese_rows.csv'
remaining_japanese_rows.to_csv(output_file, index=False)

# Notify the user about the saved file
if remaining_japanese_count > 0:
    print(f"\nRemaining rows saved to: {output_file}")
else:
    print("\nNo rows labeled as 'Japanese' remain.")

Rows still labeled as 'Japanese': 3

Details of rows still labeled as 'Japanese':
 Ranking                                          Name                                                                                                                                                           URL  Rating                     Price Range  Review Count Categories                                                    Address  Latitude  Longitude
     103                           Kyubey Ginza Honten                      https://www.tripadvisor.com/Restaurant_Review-g14129573-d803208-Reviews-Kyubey_Ginza_Honten-Ginza_Chuo_Tokyo_Tokyo_Prefecture_Kanto.html     NaN 4.44.4 of 5 bubbles(905reviews)           NaN   Japanese               8-7-6, Ginza, Chuo 104-0061 Tokyo Prefecture 35.668510 139.761213
     104                       Tokyo Shiba Tofuya Ukai           https://www.tripadvisor.com/Restaurant_Review-g14129730-d1676624-Reviews-Tokyo_Shiba_Tofuya_Ukai-Shibakoen_Minato_Tokyo_Tokyo_Prefecture_

In [41]:
tabelog_data = pd.read_csv('tabelog_data_cleaned.csv')
tripadvisor_data = pd.read_csv('trip_advisor_data_cleaned.csv')

# Step 1: Count and print remaining rows labeled 'Japanese' in tripadvisor_data
remaining_japanese_rows = tripadvisor_data[tripadvisor_data['Categories'] == 'Japanese']
remaining_japanese_count = remaining_japanese_rows.shape[0]

# Print the count and details of remaining 'Japanese' rows
print(f"Rows still labeled as 'Japanese': {remaining_japanese_count}")
if remaining_japanese_count > 0:
    print("\nDetails of rows still labeled as 'Japanese':")
    print(remaining_japanese_rows.to_string(index=False))  # Print rows without the index
    
# Save remaining 'Japanese' rows to a file (optional)
if remaining_japanese_count > 0:
    remaining_japanese_rows.to_csv('remaining_japanese_tripadvisor_rows.csv', index=False)
    print(f"\nRemaining rows saved to: 'remaining_japanese_tripadvisor_rows.csv'")
else:
    print("\nNo rows labeled as 'Japanese' remain in tripadvisor_data_cleaned.csv.")

# Step 2: Group by 'Categories' and count rows in both datasets
tabelog_grouped = tabelog_data.groupby('Categories').size().reset_index(name='Tabelog_Count')
tripadvisor_grouped = tripadvisor_data.groupby('Categories').size().reset_index(name='Tripadvisor_Count')

# Step 3: Merge grouped data from both datasets
combined_cleaned = pd.merge(
    tabelog_grouped,
    tripadvisor_grouped,
    on='Categories',
    how='outer'
)

# Step 4: Handle missing values and calculate total count
combined_cleaned.fillna(0, inplace=True)  # Replace NaN with 0
combined_cleaned['Total_Count'] = combined_cleaned['Tabelog_Count'] + combined_cleaned['Tripadvisor_Count']

# Step 5: Sort by total count in descending order
combined_cleaned = combined_cleaned.sort_values(by='Total_Count', ascending=False)

# Print the combined data
print("\nCombined and sorted category counts:")
print(combined_cleaned)

# Step 6: Save the combined data to a CSV file
combined_cleaned.to_csv('combined_cleaned_category_counts.csv', index=False)
print("\nCombined category counts saved to: 'combined_cleaned_category_counts.csv'")

Rows still labeled as 'Japanese': 0

No rows labeled as 'Japanese' remain in tripadvisor_data_cleaned.csv.

Combined and sorted category counts:
                Categories  Tabelog_Count  Tripadvisor_Count  Total_Count
51                   Sushi           43.0               19.0         62.0
34                 Kaiseki           41.0                2.0         43.0
60                Yakiniku           13.0               18.0         31.0
19                  French           27.0                2.0         29.0
1                      Bar            1.0               16.0         17.0
..                     ...            ...                ...          ...
44         Sichuan Cuisine            1.0                0.0          1.0
27  Houseboat and cruising            1.0                0.0          1.0
6                    Bread            1.0                0.0          1.0
5                   Bistro            1.0                0.0          1.0
31                   Irish            0.0